In [1]:
import xml.etree.ElementTree as ET
import os
import re
from openpyxl import Workbook
import src

%load_ext autoreload
%autoreload 2

In [2]:
# Parsing the Design Pattern List XML file (provided with P-Mart repo)
tree = ET.parse("Design Pattern List v1.2.xml")
root = tree.getroot()

In [3]:
ET.indent(root)
print(ET.tostring(root, encoding='unicode'))

<designPatterns>
  <program type="Java">
    <name>1 - QuickUML 2001</name>
    <designPattern name="Abstract Factory">
      <microArchitectures>
        <microArchitecture number="1">
          <roles>
            <clients>
              <client roleKind="AbstractClass">
                <entity>diagram.DefaultFigureEditor</entity>
              </client>
              <client roleKind="AbstractClass">
                <entity>diagram.DiagramUI</entity>
              </client>
            </clients>
            <abstractProducts>
              <abstractProduct roleKind="AbstractClass">
                <entity>diagram.FigureRenderer</entity>
              </abstractProduct>
            </abstractProducts>
            <products>
              <product roleKind="Class">
                <entity>uml.diagram.ClassRenderer</entity>
              </product>
              <product roleKind="Class">
                <entity>uml.diagram.AssociationLinkRenderer</entity>
              </product>
   

In [4]:
root.tag

'designPatterns'

In [5]:
for i in root.iter():
    if i.attrib.get("name", ) == "Adapter":
        print(i.tag, i.attrib)

designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}
designPattern {'name': 'Adapter'}


In [6]:
for element in root:
    print(element.tag)
    for el in element:
        print(el.tag)

program
name
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
program
name
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
designPattern
p

In [7]:
singleton_patterns = []

for program in root:
    # Extract all singleton instances
    src_code_name = None
    for element in program:
        if element.tag=="name":
            src_code_name = element.text
        if element.attrib.get("name") == "Singleton":
            singleton_patterns.append((src_code_name, element))

In [8]:
singleton_files = []

# Extract project names and filenames that contain a singleton design pattern
for src_code_name, singleton in singleton_patterns:
    for element in singleton.iter():
        if element.tag == "singleton":
            for entity in element.iter():
                if entity.tag=="entity":
                    singleton_files.append((str(src_code_name), str(entity.text)))
                    print('{:<30} - {:>5}'.format(src_code_name, entity.text))

1 - QuickUML 2001              - uml.ui.IconManager
2 - Lexi v0.1.1 alpha          - com.jmonkey.export.Registry
2 - Lexi v0.1.1 alpha          - com.jmonkey.office.lexi.support.EditorActionManager
3 - JRefactory v2.6.24         - org.acm.seguin.uml.loader.ReloaderSingleton
3 - JRefactory v2.6.24         - org.acm.seguin.ide.jbuilder.UMLNodeViewerFactory
5 - JUnit v3.7                 - junit.framework.Assert
5 - JUnit v3.7                 - junit.runner.Version
6 - JHotDraw v5.1              - CH.ifa.draw.util.Clipboard
6 - JHotDraw v5.1              - CH.ifa.draw.util.Iconkit
8 - MapperXML v1.9.7           - com.taursys.debug.Debug
8 - MapperXML v1.9.7           - com.taursys.html.HTMLComponentFactory
8 - MapperXML v1.9.7           - com.taursys.tools.CodeGenerator
10 - Nutch v0.4                - net.nutch.io.NullWritable
13 - Software architecture design patterns in Java - src.SINGLETON.FileLogger
14 - DrJava v20020619          - edu.rice.cs.drjava.model.definition.indent.Identer
1

In [9]:
def generate_singleton_paths():
    """
    Generates the singleton filepaths one by one
    """
    for project_name, filename in singleton_files:
        if project_name in os.listdir("source-codes"):
            filepath_from_src = filename.replace(".", "/")
            rel_filepath = os.path.join("source-codes", project_name, "src", filepath_from_src)
            yield project_name, rel_filepath

In [10]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE | re.DOTALL)

    def _replacer(match):
        # if the 2nd group (capturing comments) is not None,
        # it means we have captured a non-quoted (real) comment string.
        if match.group(2) is not None:
            return ""  # so we will return empty to remove the comment
        else:  # otherwise, we will return the 1st group
            return match.group(1)  # captured quoted-string

    return regex.sub(_replacer, string)

In [11]:
remove_comments('''package junit.ui;


 
/**
 * @deprecated use junit.awtui.TestRunner.
 */
 public class TestRunner extends junit.awtui.TestRunner {
	public static void main(String[] args) {
		new TestRunner().start(args);
	}
}''')

'package junit.ui;\n\n\n\n\n public class TestRunner extends junit.awtui.TestRunner {\n\tpublic static void main(String[] args) {\n\t\tnew TestRunner().start(args);\n\t}\n}'

In [12]:
# # Prompt Generator and Excel sheet creator
# path_generator = generate_singleton_paths()
# wb = Workbook()
# ws = wb.active
# ws.title = "GPT-5 Outputs"

# prompt_base = None
# with open("prompt.txt", "r") as prompt_file:
#     prompt_base = prompt_file.read()
#     prompt_file.close()

# for project_name, rel_filepath in path_generator:
#     code = None
#     with open(rel_filepath+".java", "r") as input_code_file:
#         code = input_code_file.read()
#     code_stripped_cmnts = remove_comments(code).strip()
    
#     # Creating the Excel sheet for storing the responses
#     row_values = [project_name, os.path.basename(rel_filepath)]
#     ws.append(row_values)
    
#     # Writing prompts to prompt files
#     prompt_output_path = os.path.join("./prompts-engineered", project_name+ "-" + os.path.basename(rel_filepath) + ".txt")
#     with open(prompt_output_path, "w") as prompt_output_file:
#         prompt_output_file.write(prompt_base.format(code=code_stripped_cmnts))
#         prompt_output_file.close()

# wb.save('test-outputs/singleton-outputs.xlsx')

In [13]:
str(None)

'None'

In [14]:
type(root)

xml.etree.ElementTree.Element

In [9]:
import src.prompt_generation as PG

In [39]:
from itertools import product

patterns = ["adapter", "singleton", "decorator", "facade", "flyweight", "bridge", "composite", "proxy"]
wrong = [True, False]
prompts = [0, 1]

for i in product(patterns, wrong, prompts):
    print(i)
    PG.generate_prompt_files(root, i[0], i[1], False, i[2])

('adapter', True, 0)
3 - JRefactory v2.6.24 adapter source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/awt/Question
source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/awt/Question
3 - JRefactory v2.6.24 adapter source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/uml/refactor/VariableListCellRenderer
source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/uml/refactor/VariableListCellRenderer
3 - JRefactory v2.6.24 adapter source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/parser/ast/ASTArrayDimsAndInits
source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/parser/ast/ASTArrayDimsAndInits
3 - JRefactory v2.6.24 adapter source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/pretty/LocalVariableLookAhead
source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/pretty/LocalVariableLookAhead
3 - JRefactory v2.6.24 adapter source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/ide/cafe/CafeExtractMethod
source-codes/3 - JRefactory v2.6.24/src/org/acm/seguin/ide/cafe/CafeExtractM

In [21]:
file = open("source-codes/11 - PMD v1.8/net/sourceforge/pmd/cpd/PHPTokenizer.java", "r")
file.close()

In [18]:
os.path.dirname("wow/hellow/hello")

'wow/hellow'

In [5]:
import src.api.OpenrouterAPI as ORapi
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [8]:
ORapi.run_model(
    model_name="qwen/qwen-2.5-coder-32b-instruct:free",
    pattern="singleton",
    prompt_type=2,
    correct=False,
)
ORapi.run_model(
    model_name="qwen/qwen-2.5-coder-32b-instruct:free",
    pattern="singleton",
    prompt_type=2,
    correct=True,
)
ORapi.run_model(
    model_name="qwen/qwen-2.5-coder-32b-instruct:free",
    pattern="adapter",
    prompt_type=2,
    correct=False,
)
ORapi.run_model(
    model_name="qwen/qwen-2.5-coder-32b-instruct:free",
    pattern="adapter",
    prompt_type=2,
    correct=True,
)

Loaded prompt 1 - QuickUML 2001- MetaMethod.txt....
Request sent to qwen/qwen-2.5-coder-32b-instruct:free....
Response Received.... Writing to summary-outputs/qwen/qwen-2.5-coder-32b-instruct/incorrect/singleton/singleton/1 - QuickUML 2001- MetaMethod.txt
Error Message Received and Logged... Aborting


SystemExit: 1

/home/oishik/repos/Detecting-SDP/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
import src.api.LocalInference as LocalInference

In [ ]:
model_id = "rjmalagon/Nxcode-CQ-7B-orpo-Q8_0-GGUF"

LocalInference.run_model(model_id=model_id, pattern="singleton", correct=True)

llama_model_loader: loaded meta data with 24 key-value pairs and 387 tensors from /home/oishik/.cache/huggingface/hub/models--rjmalagon--Nxcode-CQ-7B-orpo-Q8_0-GGUF/snapshots/c275a6ab995ff30e018470d9a27c872d5e861e4a/nxcode-cq-7b-orpo.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Nxcode-CQ-7B-orpo
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 32
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 65536
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 13440
llama_model_loader: - kv   6:      

Prompt 1 - QuickUML 2001- IconManager.txt already used..... Skipping
Prompt 2 - Lexi v0.1.1 alpha- EditorActionManager.txt already used..... Skipping
Loaded prompt 2 - Lexi v0.1.1 alpha- Registry.txt....
Model inferenced...


ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/oishik/repos/Detecting-SDP/.venv/lib/python3.13/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
    ~~~~~~~~^^
  File "/home/oishik/repos/Detecting-SDP/.venv/lib/python3.13/site-packages/ipykernel/kernelbase.py", line 701, in shell_main
    await self.dispatch_shell(msg, subshell_id=subshell_id)
  File "/home/oishik/repos/Detecting-SDP/.venv/lib/python3.13/site-packages/ipykernel/kernelbase.py", line 484, in dispatch_shell
    sys.stderr.flush()
    ~~~~~~~~~~~~~~~~^^
ValueError: I/O operation on closed file.


In [2]:
import src.utils as utils

In [3]:
utils.evaluate_files("qwen/qwen-2.5-coder-32b-instruct:free")

llama_context: n_ctx_per_seq (2046) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


FileNotFoundError: [Errno 2] No such file or directory: 'test-outputs/qwen/qwen-2.5-coder-32b-instruct:free'